## Data preprocessing

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import librosa

In [27]:
import os
import shutil

In [28]:
from datasets import load_dataset, Dataset, Audio, DatasetDict

In [34]:
df = pd.read_csv('labels.txt', sep='\t')

In [35]:
df = df.rename(columns={'FILE': 'file', 'TRANSCRIPT': 'text'})

In [36]:
df['id'] = df["file"].apply(lambda x: x.split('\\')[-1].split('.')[0])

In [37]:
def transform_subset(row):
    utt_id = int(row['ID'])
    if utt_id <= 402:
        return 'train'
    elif utt_id >= 403 and utt_id <= 427:
        return 'dev'
    elif utt_id >= 428 and utt_id <= 452:
        return 'test'

In [38]:
df['SUBSET'] = df.apply(transform_subset, axis=1)

In [39]:
df

,file,SPEAKER,GENDER,AGE,LANGUAGES,ACCENT,MODE,SUBSET,ID,text,id
0,nist\TEST\whisper\US\101\s101u011w.WAV,101,F,15-20,"EN,SP",US,whisper,train,11,He will allow a rare lie.,s101u011w
1,nist\TEST\whisper\US\101\s101u017w.WAV,101,F,15-20,"EN,SP",US,whisper,train,17,Carl lives in a lively home.,s101u017w
2,nist\TEST\whisper\US\101\s101u020w.WAV,101,F,15-20,"EN,SP",US,whisper,train,20,"She wore warm, fleecy, woolen overalls.",s101u020w
3,nist\TEST\whisper\US\101\s101u023w.WAV,101,F,15-20,"EN,SP",US,whisper,train,23,Those musicians harmonize marvelously.,s101u023w
4,nist\TEST\whisper\US\101\s101u025w.WAV,101,F,15-20,"EN,SP",US,whisper,train,25,Only lawyers love millionaires.,s101u025w
...,...,...,...,...,...,...,...,...,...,...,...
42122,nist\TRAIN\normal\SG\019\s019u448n.WAV,19,F,25-30,"EN,MA",SG,normal,test,448,Spherical gifts are difficult to wrap.,s019u448n
42123,nist\TRAIN\normal\SG\019\s019u449n.WAV,19,F,25-30,"EN,MA",SG,normal,test,449,Ralph prepared red snapper with fresh lemon sa...,s019u449n
42124,nist\TRAIN\normal\SG\019\s019u450n.WAV,19,F,25-30,"EN,MA",SG,normal,test,450,Roy ignored the spurious data points in drawin...,s019u450n
42125,nist\TRAIN\normal\SG\019\s019u451n.WAV,19,F,25-30,"EN,MA",SG,normal,test,451,The thick elm forest was nearly overwhelmed by...,s019u451n


In [13]:
df_an = pd.read_csv('labels.txt', sep='\t')

In [15]:
df_an['id'] = df_an["FILE"].apply(lambda x: x.split('\\')[-1].split('.')[0])

In [17]:
z = []
for _ in df_an['id'].to_list():
    if _ not in df['id'].to_list():
        z.append(_)
print(sorted(z))

['s002u267w', 's002u289w', 's002u291w', 's003u117w', 's101u003n', 's101u004n', 's101u005n', 's101u005w', 's101u006n', 's101u006w', 's101u007n', 's101u007w', 's101u008n', 's101u008w', 's101u009n', 's101u009w', 's101u010n', 's101u010w', 's101u011n', 's101u011w', 's101u012n', 's101u012w', 's101u013w', 's101u014w', 's101u015w', 's101u016w', 's101u017w', 's101u018w', 's101u019w', 's101u020w', 's101u021w', 's101u022w', 's101u023w', 's101u024w', 's101u025w', 's101u026w', 's101u027w', 's101u028w', 's101u029w', 's101u030w', 's101u031w', 's101u032w', 's101u033w', 's101u034w', 's104u003w', 's104u004w', 's104u005w', 's104u006w', 's104u007w', 's104u008n', 's104u008w', 's104u009w', 's104u010w', 's104u011w', 's104u012w', 's104u013w', 's104u014w', 's104u015w', 's104u016w', 's104u017w', 's104u018w', 's104u019w', 's104u020w', 's104u021w', 's104u022w', 's104u023w', 's104u024w', 's104u025w', 's104u027w', 's104u029w', 's104u030w', 's104u031w', 's104u032w', 's104u033w', 's104u035n', 's104u035w', 's104u037w'

In [21]:
print(len([_ for _ in z if _.split('u')[0].split('s')[1] != '104']))

46


In [24]:
len(df[df['SUBSET'] == 'train'])

37172

In [25]:
len(df[df['SUBSET'] == 'dev'])

2250

In [26]:
len(df[df['SUBSET'] == 'test'])

2233

In [15]:
for i, row in df.iterrows():
    shutil.move(row['file'], 'audio/{}/{}.WAV'.format(row['SUBSET'].lower(), row['id']))

In [40]:
main_directory = 'audio'

def get_file_id(filename):
    return os.path.splitext(filename)[0]

for root, dirs, files in os.walk(main_directory):
    for file in files:
        file_id = get_file_id(file)
        if file_id not in ids:
            file_path = os.path.join(root, file)
            print(f"Deleting file: {file_path}")
            os.remove(file_path)

print("Cleanup completed.")

Cleanup completed.


In [18]:
ids = df['id'].to_list()

In [21]:
with open('ids.txt', 'w', encoding='utf-8') as f:
    for _ in ids:
        f.write('{}\n'.format(_))

In [11]:
def collect_files(folder_path):
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

main_folder = "FA_output"
files = collect_files(main_folder)
print(len(files))

41621


In [12]:
files = [x.split('\\')[-1].split('.')[0] for x in files]

In [13]:
n_files = df[df['SUBSET'] == 'train']['id'].tolist()

In [14]:
missin = []
for _ in n_files:
    if _ not in files:
        missin.append(_)

In [25]:
print(missin)

['s104u052w', 's104u059w', 's104u060w', 's104u066w', 's104u087w', 's104u120w', 's104u148w', 's104u176w', 's104u219w', 's104u240w', 's104u299w', 's104u306w', 's104u352w', 's104u372w', 's104u395w', 's104u396w', 's003u115w', 's104u026w', 's104u028w', 's104u036w', 's104u079w', 's104u088w', 's104u097w', 's104u179w', 's104u218w', 's104u228w', 's104u259w', 's104u260w', 's104u291w', 's104u297w', 's104u301w', 's104u309w']


In [27]:
source_dir = 'audio/train'
target_dir = 'audio/throw'

for file in missin:
    source_path = os.path.join(source_dir, file+'.WAV')
    target_path = os.path.join(target_dir, file+'.WAV')
    
    if os.path.exists(source_path):
        shutil.move(source_path, target_path)
        print(f'Moved: {file}')
    else:
        print(f'File not found: {file}')

File not found: s104u052w
File not found: s104u059w
Moved: s104u060w
Moved: s104u066w
Moved: s104u087w
Moved: s104u120w
Moved: s104u148w
Moved: s104u176w
Moved: s104u219w
Moved: s104u240w
Moved: s104u299w
Moved: s104u306w
Moved: s104u352w
Moved: s104u372w
Moved: s104u395w
Moved: s104u396w
Moved: s003u115w
Moved: s104u026w
Moved: s104u028w
Moved: s104u036w
Moved: s104u079w
Moved: s104u088w
Moved: s104u097w
Moved: s104u179w
Moved: s104u218w
Moved: s104u228w
Moved: s104u259w
Moved: s104u260w
Moved: s104u291w
Moved: s104u297w
Moved: s104u301w
Moved: s104u309w


In [4]:
import os

def count_files_in_folders(directory):
    folders = [f.path for f in os.scandir(directory) if f.is_dir()]
    sums = 0
    
    for folder in folders:
        file_count = len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])
        sums += file_count
        print(f"Folder '{os.path.basename(folder)}' contains {file_count} files.")
    print(sums)

directory_path = 'PM_align'
count_files_in_folders(directory_path)

Folder 's000' contains 900 files.
Folder 's001' contains 900 files.
Folder 's002' contains 897 files.
Folder 's003' contains 898 files.
Folder 's004' contains 899 files.
Folder 's005' contains 900 files.
Folder 's006' contains 900 files.
Folder 's007' contains 900 files.
Folder 's008' contains 900 files.
Folder 's009' contains 900 files.
Folder 's010' contains 900 files.
Folder 's011' contains 900 files.
Folder 's012' contains 899 files.
Folder 's013' contains 900 files.
Folder 's014' contains 900 files.
Folder 's015' contains 900 files.
Folder 's016' contains 900 files.
Folder 's017' contains 900 files.
Folder 's018' contains 900 files.
Folder 's019' contains 900 files.
Folder 's101' contains 876 files.
Folder 's102' contains 900 files.
Folder 's103' contains 899 files.
Folder 's104' contains 751 files.
Folder 's105' contains 900 files.
Folder 's106' contains 900 files.
Folder 's107' contains 900 files.
Folder 's108' contains 900 files.
Folder 's109' contains 900 files.
Folder 's110' 

In [10]:
for index, row in df.iterrows():
    with open('labs/{}.lab'.format(row['id']), 'w', encoding='utf-8') as f:
        f.write(row['text'].replace('.', ''))

In [10]:
def transform_file(row):
    return f"audio/{row['SUBSET'].lower()}/{row['id']}.WAV"

In [11]:
df['file'] = df.apply(transform_file, axis=1)